[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ffserro/MVP/blob/master/mvp.ipynb)


# Regressão Linear para Series Temporais - Planejamento dos dispêndios de alimentação de militares da Marinha do Brasil

## Contextualização



<div align="justify" font-size="40px">
O planejamento eficiente dos recursos logísticos é um dos pilares para a manutenção da prontidão e da capacidade operacional das Forças Armadas. Entre os diversos insumos estratégicos, a alimentação das organizações militares desempenha papel central, tanto no aspecto orçamentário quanto no suporte direto às atividades diárias. Na Marinha do Brasil, a gestão dos estoques e dos gastos com gêneros alimentícios envolve múltiplos órgãos e abrange um volume expressivo de transações financeiras e contábeis, tornando-se um processo complexo e suscetível a variações sazonais, econômicas e administrativas.

Neste cenário, prever com maior precisão os custos relacionados ao consumo de alimentos é fundamental para otimizar a alocação de recursos públicos, reduzir desperdícios, evitar rupturas de estoque e aumentar a eficiência do planejamento orçamentário. Tradicionalmente, esse processo é conduzido por meio de análises históricas e técnicas de planejamento administrativo. No entanto, tais abordagens muitas vezes não capturam adequadamente os padrões temporais e as variáveis externas que influenciam os gastos.

A ciência de dados, e em particular as técnicas de modelagem de séries temporais, surge como uma alternativa poderosa para aprimorar esse processo decisório. Modelos como SARIMA, Prophet, XGBoost e LSTM permitem identificar tendências, sazonalidades e anomalias nos dados, possibilitando não apenas previsões mais robustas, mas também a geração de insights que subsidiam políticas de abastecimento e aquisição.

Assim, o presente trabalho propõe a aplicação de técnicas de análise e previsão de séries temporais sobre os dados históricos de consumo de alimentos da Marinha do Brasil, com o objetivo de estimar os custos futuros e explorar padrões relevantes que possam apoiar o processo de gestão logística e orçamentária. A relevância deste estudo reside não apenas no ganho potencial de eficiência administrativa, mas também na contribuição para a transparência, a racionalização do gasto público e a modernização da gestão de suprimentos em instituições estratégicas para o país.
</div>

## Glossário


* Municiamento
* Rancho
* Etapa
* Comensal
* Série Temporal
* Tendência
* Sazonalidade
* Estacionariedade


## Trabalho

In [ ]:
#@title Download dos dados
!git clone 'https://github.com/ffserro/MVP.git'
!pip install -r '/content/MVP/requirements.txt' > '/content/pip_log.txt'
!if grep -iq "downloading" '/content/pip_log.txt';then python -c "import os;print('Por favor, reinicie a sessão e execute novamente.');os.kill(os.getpid(), 9)"; else python -c "print('Vamos começar!')"; fi

fatal: destination path 'MVP' already exists and is not an empty directory.
Vamos começar!


In [ ]:
#@title Import de bibliotecas
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


import pmdarima as pm
from prophet import Prophet
from xgboost import XGBRegressor

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
mmm = pd.DataFrame()
mmm = pd.concat([mmm]+[pd.read_excel(d) for d in glob('/content/MVP/dados/mmm/*')]).reset_index(drop=True)

etapas = pd.DataFrame()
etapas = pd.concat([etapas]+[pd.read_excel(d) for d in glob('/content/MVP/dados/etapas/*')]).reset_index(drop=True).rename(columns={'uasg':'codigo'})

da = pd.DataFrame()
da = pd.concat([da]+[pd.read_excel(d) for d in glob('/content/MVP/dados/da/*')]).reset_index(drop=True)

In [ ]:
mmm_marinha = mmm.groupby(['ano', 'mes'])[[col for col in mmm.columns if col not in ['ano', 'mes', 'codigo', 'nome']]].sum().reset_index()

In [ ]:
mmm_marinha['mes'] = [''.join([str(i[0]), '_', str(i[1])]) for i in zip(mmm_marinha.mes.values, mmm_marinha.ano.values)]
mmm_marinha.drop(columns=['ano'], inplace=True)
mmm_marinha = mmm_marinha.iloc[:-2]

In [ ]:
mmm_receita_despesa = pd.merge(mmm, da, how='inner', on=['mes', 'ano', 'codigo'])

In [ ]:
mmm_etapas = pd.merge(left=mmm, right=etapas, how='inner', on=['ano', 'mes', 'codigo'])

In [ ]:
mmm_etapas[mmm_etapas.codigo_etapa.isin([103, 105])][['ano', 'mes', 'nome', 'codigo_etapa', 'quantidade']]

,ano,mes,nome,codigo_etapa,quantidade
1,2022,1,1° BATALHAO DE OPERAÇOES LITORANEAS DE FUZILEI...,103,18644.0
3,2022,1,1º Batalhão de Operações Ribeirinhas,103,11735.0
9,2022,1,2° BATALHÃO DE OPERAÇÕES LITORÂNEAS DE FUZILEI...,103,5676.0
10,2022,1,2° BATALHÃO DE OPERAÇÕES LITORÂNEAS DE FUZILEI...,105,40.0
12,2022,1,2º BATALHÃO DE OPERAÇÕES RIBEIRINHAS,103,6946.0
...,...,...,...,...,...
60067,2020,12,SUBMARINO TUPI,103,613.0
60071,2020,12,TRIBUNAL MARÍTIMO,103,751.0
60072,2020,12,TRIBUNAL MARÍTIMO,105,291.0
60076,2020,12,UNIDADE INTEGRADA DE SAUDE MENTAL,103,3296.0


In [ ]:
pd.DataFrame({
    'diferenca': mmm_receita_despesa.despesa_autorizada_global - (mmm_receita_despesa.generos_consumidos + mmm_receita_despesa.vale_extra),
    'sobra_licita': mmm_receita_despesa.sobra_licita
})

,diferenca,sobra_licita
0,35427.6200,35427.62
1,35780.1180,80008.51
2,22410.1814,22410.17
3,36713.9260,36713.88
4,-92672.4740,0.00
...,...,...
14122,7978.8400,7978.82
14123,4670.4700,4670.46
14124,35.2000,35.20
14125,20742.2000,8039.14


In [ ]:
da_marinha = da.groupby(['mes', 'ano']).despesa_autorizada_global.sum().reset_index()
da_marinha['mes'] = ['_'.join(['{:02d}'.format(i[0]), str(i[1])]) for i in zip(da_marinha.mes.values, da_marinha.ano.values)]
da_marinha = da_marinha.sort_values(by=['ano', 'mes']).iloc[:-2]
fig = px.line(
    da_marinha,
    x = 'mes',
    y = 'despesa_autorizada_global',
    title='Despesa Autorizada global'
)

fig.show()

In [ ]:
def grafico_base(titulo):
    return px.line(
        mmm_marinha,
        x = 'mes',
        y = 'totais_balanco_paiol_despesa',
        labels = {
            'mes': 'Mês e ano',
            'totais_balanco_paiol_despesa': 'Totais das despesas'
        },
        title = titulo
    )

fig = grafico_base('Gastos com alimentação dos últimos cinco anos')

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Teste de estacionariedade

result = adfuller(mmm_marinha.totais_balanco_paiol_despesa)
print(f'ADF: {result[0]}, p-valor: {result[1]}')

ADF: -0.792984009986987, p-valor: 0.8211742792131476


In [ ]:
naive_forecast = mmm_marinha.totais_balanco_paiol_despesa.shift(1)
mae_naive = mean_absolute_error(mmm_marinha.totais_balanco_paiol_despesa.iloc[1:], naive_forecast.iloc[1:])
print('Baseline Naïve MAE:', mae_naive)

Baseline Naïve MAE: 1371822.7820545628


In [ ]:
# SARIMA

train = mmm_marinha.iloc[:-12, :]
test = mmm_marinha.iloc[-12:, :]

model_auto = pm.auto_arima(train.totais_balanco_paiol_despesa, seasonal=True, m=12, stepwise=True, trace=True)
print(model_auto.summary())

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=2075.426, Time=2.91 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=2081.293, Time=0.06 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=2071.938, Time=0.34 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=2073.317, Time=0.37 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=2079.434, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=2077.323, Time=0.07 sec
 ARIMA(1,1,0)(2,0,0)[12] intercept   : AIC=2073.498, Time=1.02 sec
 ARIMA(1,1,0)(1,0,1)[12] intercept   : AIC=2072.305, Time=0.72 sec
 ARIMA(1,1,0)(0,0,1)[12] intercept   : AIC=2072.411, Time=0.19 sec
 ARIMA(1,1,0)(2,0,1)[12] intercept   : AIC=2074.220, Time=1.53 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=2073.821, Time=0.54 sec
 ARIMA(2,1,0)(1,0,0)[12] intercept   : AIC=2073.333, Time=2.26 sec
 ARIMA(1,1,1)(1,0,0)[12] intercept   : AIC=2073.515, Time=0.67 sec
 ARIMA(0,1,1)(1,0,0)[12] intercept   : AIC=2072.829, Time=0.25 sec
 ARIMA(2,1,1)(1,0,0

In [ ]:
mmm

,ano,mes,codigo,nome,balanco_paiol_mes_anterior,gen_depsubmrj_depnav_reg,gen_adq_form_extra_mb_licit1,gen_adq_form_extra_mb_licit2,gen_adq_form_extra_mb_slicit,remessa_recebida,remessa_expedida,vale_extra,termo_de_despesa,generos_consumidos,totais_balanco_paiol_receita,totais_balanco_paiol_despesa,saldo
0,2022,1,81200,1° BATALHAO DE OPERAÇOES LITORANEAS DE FUZILEI...,109570.962500,159243.58,31968.7400,0.00,3220.8400,0.00,0.0,1767.81,0.0,176446.5700,304004.122500,178214.3800,125789.742500
1,2022,1,88200,1º Batalhão de Operações Ribeirinhas,213997.273380,137689.49,24477.5400,0.00,12914.5000,0.00,0.0,11069.42,0.0,160103.9620,389078.803380,171173.3820,217905.421380
2,2022,1,82200,2° BATALHÃO DE OPERAÇÕES LITORÂNEAS DE FUZILEI...,268711.846000,25676.39,5036.9000,0.00,400.0000,0.00,0.0,5552.64,0.0,77699.1786,299825.136000,83251.8186,216573.317400
3,2022,1,84200,2º BATALHÃO DE OPERAÇÕES RIBEIRINHAS,321259.222500,3522.05,4163.1000,0.00,0.0000,0.00,0.0,0.00,0.0,92672.4740,328944.372500,92672.4740,236271.898500
4,2022,1,83200,3° BATALHÃO DE OPERAÇÕES LITORÂNEAS DE FUZILEI...,127460.105700,36189.24,83347.4112,1767.10,16343.5000,14342.28,0.0,2628.16,0.0,133555.7800,279449.636900,136183.9400,143265.696900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12553,2020,12,91531,SUBMARINO TAMOIO,13226.001940,2075.00,4071.2885,0.00,3685.2866,0.00,0.0,0.00,0.0,16267.1600,23057.577040,16267.1600,6790.417040
12554,2020,12,91530,SUBMARINO TUPI,28318.016913,0.00,93.0000,0.00,8248.9953,0.00,0.0,0.00,0.0,25102.7300,36660.012213,25102.7300,11557.282213
12555,2020,12,91535,Submarino Riachuelo,0.000000,0.00,0.0000,0.00,0.0000,0.00,0.0,0.00,0.0,0.0000,0.000000,0.0000,0.000000
12556,2020,12,10100,TRIBUNAL MARÍTIMO,9205.470000,0.00,0.0000,16234.06,0.0000,0.00,0.0,0.00,0.0,9848.8000,25439.530000,9848.8000,15590.730000


In [ ]:
sarima_model = SARIMAX(train.totais_balanco_paiol_despesa,
                       order=model_auto.order,
                       seasonal_order=model_auto.seasonal_order)

sarima_fit = sarima_model.fit(disp=False)

forecast_sarima = sarima_fit.get_forecast(steps=12)
pred_sarima = forecast_sarima.predicted_mean

mae_sarima = mean_absolute_error(test.totais_balanco_paiol_despesa, pred_sarima)
print('SARIMA MAE:', mae_sarima)



SARIMA MAE: 1582296.513483627


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo SARIMA')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_sarima
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Previsão com o Prophet

prophet_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
prophet_df.columns = ['ds', 'y']
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'], format='%m_%Y')

model_prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
model_prophet.fit(prophet_df)

future = model_prophet.make_future_dataframe(periods=12, freq='M')
forecast = model_prophet.predict(future)

forecast_test = forecast.set_index('ds').loc[pd.to_datetime(test.mes, format='%m_%Y')]
mae_prophet = mean_absolute_error(test['totais_balanco_paiol_despesa'], forecast_test['yhat'])
print('Prophet MAE', mae_prophet)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpwrdcoy8z/op21y2jt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpwrdcoy8z/y10y9e16.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=46287', 'data', 'file=/tmp/tmpwrdcoy8z/op21y2jt.json', 'init=/tmp/tmpwrdcoy8z/y10y9e16.json', 'output', 'file=/tmp/tmpwrdcoy8z/prophet_modelx4zcef86/prophet_model-20250903233140.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
23:31:40 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:31:41 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Prophet MAE 1378543.2569372256


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo Prophet')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = forecast_test.yhat
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
# Regressão com XGBoost para séries temporais

xg_df = mmm_marinha[['mes', 'totais_balanco_paiol_despesa']]
xg_df['ano'] = xg_df['mes'].apply(lambda x: x.split('_')[-1]).astype(int)
xg_df['mes'] = xg_df['mes'].apply(lambda x: x.split('_')[0]).astype(int)
xg_df['lag1'] = xg_df['totais_balanco_paiol_despesa'].shift(1)
xg_df['lag3'] = xg_df['totais_balanco_paiol_despesa'].shift(3)
xg_df['lag6'] = xg_df['totais_balanco_paiol_despesa'].shift(6)
xg_df['rolling3'] = xg_df['totais_balanco_paiol_despesa'].rolling(3).mean()
xg_df['rolling6'] = xg_df['totais_balanco_paiol_despesa'].rolling(6).mean()

xg_df = xg_df.dropna().reset_index(drop=True)

In [ ]:
train_xg = xg_df.iloc[:-12]
test_xg = xg_df.iloc[-12:]

X_train = train_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_train = train_xg['totais_balanco_paiol_despesa']
X_test = test_xg.drop(columns=['totais_balanco_paiol_despesa'])
y_test = test_xg['totais_balanco_paiol_despesa']

xgb = XGBRegressor(
    n_estimators = 300,
    learning_rate = 0.05,
    max_depth = 5,
    subsample = 0.8,
    colsample_bytree = 0.8,
    random_state = 42
)

xgb.fit(X_train, y_train)

pred_xgb = xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, pred_xgb)

print("XGBoost MAE:", mae_xgb)


XGBoost MAE: 1848576.9457449995


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo XGBoost')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_xgb
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
hw_model = ExponentialSmoothing(
    train['totais_balanco_paiol_despesa'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

pred_hw = hw_model.forecast(12)

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo ExponentialSmoothing')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = pred_hw
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
despesas_scaled = scaler.fit_transform(mmm_marinha.totais_balanco_paiol_despesa.values.reshape(-1, 1))

def create_sequences(data, window=12):
    X, y = [], []
    for i in range(len(data)- window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_sequences(despesas_scaled)


split = len(X) - 12
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('Shape treino:', X_train.shape, y_train.shape)

Shape treino: (55, 12, 1) (55, 1)


In [ ]:
model = Sequential()
model.add(LSTM(64, activation='tanh', return_sequences=True, input_shape=(12, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='tanh'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')

history = model.fit(
    X_train, y_train,
    epochs = 200,
    batch_size = 4,
    validation_split = 0.1,
    verbose = 1
)

Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - loss: 0.2747 - val_loss: 0.1218
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1617 - val_loss: 0.1475
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1333 - val_loss: 0.0951
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1432 - val_loss: 0.0988
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1189 - val_loss: 0.0971
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1364 - val_loss: 0.0860
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1206 - val_loss: 0.1164
Epoch 8/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1147 - val_loss: 0.0924
Epoch 9/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1196 - val_loss: 0.1301
Epoch 10/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1058 - val_loss: 0.0993
Epoch 11/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1023 - val_loss: 0.1352
Epoch 12/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm = mean_absolute_error(y_test_inv, y_pred_inv)
print('LSTM MAE:', mae_lstm)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step
LSTM MAE: 1855155.3754149992


In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.reshape(1, -1)[0]
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
def create_sequences_multistep(data, window=12, horizon=12):
    X, y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        y.append(data[i+window:i+window+horizon].flatten())
    return np.array(X), np.array(y)

window = 12
horizon = 12
X, y = create_sequences_multistep(despesas_scaled, window, horizon)

split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]
print('X_train shape:', X_train.shape)
print('y_train, shape:', y_train.shape)

X_train shape: (44, 12, 1)
y_train, shape: (44, 12)


In [ ]:
model = Sequential()
model.add(LSTM(64, activation="tanh", return_sequences=True, input_shape=(window, 1)))
model.add(Dropout(0.2))
model.add(LSTM(32, activation="tanh"))
model.add(Dense(horizon))  # saída com 12 valores (multi-step)

model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=4,
    validation_split=0.1,
    verbose=1
)

Epoch 1/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - loss: 0.1818 - val_loss: 0.2179
Epoch 2/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1125 - val_loss: 0.0761
Epoch 3/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0672 - val_loss: 0.0396
Epoch 4/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0380 - val_loss: 0.0199
Epoch 5/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0403 - val_loss: 0.0563
Epoch 6/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0410 - val_loss: 0.0340
Epoch 7/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0362 - val_loss: 0.0264
Epoch 8/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0401 - val_loss: 0.0553
Epoch 9/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0360 - val_loss: 0.0298
Epoch 10/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0401 - val_loss: 0.0364
Epoch 11/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0334 - val_loss: 0.0407
Epoch 12/300
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

mae_lstm_multi = mean_absolute_error(y_test_inv.flatten(), y_pred_inv.flatten())
print('LSTM Multi-step MAE:', mae_lstm_multi)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
LSTM Multi-step MAE: 2148415.9999481947


In [ ]:
last_input_dates = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-(window + horizon):-horizon]
future_dates = pd.date_range(start = pd.to_datetime(mmm_marinha.mes, format='%m_%Y').iloc[-horizon], periods=horizon, freq='M')

In [ ]:
fig = grafico_base('Previsão temporal com o algoritmo LSTM Multi-step')

fig.add_trace(
    go.Scatter(
        x = test.mes,
        y = y_pred_inv.flatten()
    )
)

fig.update_traces(mode='lines+markers', line=dict(width=2))
fig.update_xaxes(tickangle=45)
fig.update_layout(
    template='plotly_white',
    hovermode='x unified'
    )

fig.show()

In [ ]:
teste = ['A', 'B', 'C']

In [ ]:
model = 'LSTM' #@param ["SARIMAX", "prophet", "XGBoost", "ExponentialSmothing", "LSTM"]
test = 'A' #@param

print(f"Selected model: {modelo}")
print(f'Selected test : {test}')

Selected model: SARIMAX
Selected test : A


In [ ]:
mmm['mes_ano'] = mmm.mes.astype(str) + '_' + mmm.ano.astype(str)